In [2]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import urllib
import requests 
import json

word = ['대게 맛집']
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) Gecko/20100101 Firefox/40.1'} 
서칭=[]
이름=[]
전화번호=[]
주소=[]
메뉴=[]
영업시간=[]
특징=[]
정보=[]
for i in word:
    #print(i)
    for num in range(1):#페이지수(한 페이지엔 10개의 url잇다)
        word_encode = urllib.parse.quote(i)
        driver = webdriver.Chrome('chromedriver85')
        driver.get('https://search.naver.com/search.naver?where=post&sm=tab_jum&query={}&start={}1'.format(word_encode, num))

        time.sleep(1)

        #print(word_encode)
        html = driver.page_source
        #print(html)
        bs = BeautifulSoup(html, 'html.parser') #문서 파싱 
        a = bs.findAll("a", {"class":"sh_blog_title _sp_each_url _sp_each_title"})

        count = 0

        for af in a:
            if count <10: #위에서 정한 페이지수*10
                driver.get(af['href'])
                if 'blog.naver' in af['href'] or 'blog.me' in af['href']: 

                    html = driver.page_source
                    bs = BeautifulSoup(html, 'html.parser') #문서 파싱 
                    find_url = bs.find('iframe',{"id":"mainFrame"})
                    iframe_url = find_url.get('src') #iframe URL 추출

                    con_url = 'https://blog.naver.com' + iframe_url #크롤링할 URL
                    response = requests.get(con_url, headers)
                    soup = BeautifulSoup(response.content, 'html.parser')
                    time.sleep(2)
                    map_attrs = soup.find('a',class_='se-map-info __se_link')#.attrs['data-linkdata'] #data-linkdata 여기 안에 placeId 값이 밑에 url id 값임

                    if(map_attrs != None):
                        map_attrs = map_attrs.attrs['data-linkdata'] 
                        data = json.loads(map_attrs) # data-linkdata 타입이 제이슨이라서,,
                        place_url = 'https://store.naver.com/restaurants/detail?id=' + data['placeId'] #placeId값 구한거 여기에 넣고
                        response = requests.get(place_url, headers)
                        kgy = BeautifulSoup(response.content, 'html.parser') #html 파싱하고

                        try:
                            con_data0 = kgy.find("strong", {"class":"name"}).text #맛집이름
                            이름.append(con_data0)
                        except:
                            이름.append(None)   

                        try:
                            con_data = kgy.find("div", {"class":"list_item list_item_biztel"}).text #전번
                            전화번호.append(con_data)
                        except:
                            전화번호.append(None)

                        try:
                            con_data1 = kgy.find("div", {"class":"list_item list_item_address"}).text.replace("지도보기", " ") #주소
                            주소.append(con_data1)
                        except:
                            주소.append(None)

                        try:
                            con_data2 = kgy.find("div", {"class":"list_item list_item_menu"}).text.replace("더보기", " ") #메뉴
                            메뉴.append(con_data2)
                        except:
                            메뉴.append(None)

                        try:
                            con_data3 = kgy.find("div", {"class":"list_item list_item_biztime"}).text #영업시간
                            영업시간.append(con_data3)
                        except:
                            영업시간.append(None)


                        try:
                            con_data4 = kgy.find("div", {"class":"list_item list_item_tv"}).text #특징
                            특징.append(con_data4)
                        except:
                            특징.append(None)

                        try:
                            con_data5 = kgy.find("div", {"class":"list_item list_item_convenience"}).text #정보
                            정보.append(con_data5)
                        except:
                            정보.append(None)

                        서칭.append(i)
                        count += 1
 
                    else:
                        pass
            else:
                break


In [3]:
nplace = pd.DataFrame({
    '서칭':서칭,
    '이름':이름,
    '전화번호':전화번호,
    '주소':주소,
    '메뉴':메뉴,
    '영업시간':영업시간,
    '특징':특징,
    '정보':정보  
})
# nplace

In [4]:
nplace

,서칭,이름,전화번호,주소,메뉴,영업시간,특징,정보
0,대게 맛집,강구항 대게직판장 이오,054-732-8454,경북 영덕군 강구면 강구대게길 45지번강구리 552,"변동크기 별로 상이15,000원물회80,000원대게탕 중100,000원대게탕 대",토요일 09:00 - 23:00연장 예약마지막오더 10시일요일 09:00 - 21:...,2TV생생정보 486회18.01.04. 대게코스요리,"단체석, 주차, 발렛파킹, 포장, 예약, 무선 인터넷, 반려동물 동반, 유아시설(놀..."
1,대게 맛집,착한대게,054-733-4678,경북 영덕군 강구면 강구대게길 47지번강구리 550,"변동대표영덕 대게100,000원대표모듬 회 대변동영덕 홍게변동영덕 킹크랩15,000원물회",주말 10:00 - 22:00마지막 주문 21시까지평일 10:00 - 23:00마지...,None,"단체석, 주차, 발렛파킹, 포장, 예약, 반려동물 동반"
2,대게 맛집,GS25 속초장사항점,1644-5425,강원 속초시 장사항해안길 13지번장사동 577-36,,None,None,None
3,대게 맛집,강구항영덕대게거리,None,경북 영덕군 강구면 영덕대게로 61지번강구리 350,,None,None,None
4,대게 맛집,죽도산,054-733-4148,경북 영덕군 강구면 강구대게길 47-1지번강구리 545,변동대게,주말 09:30 - 22:00평일 09:30 - 22:00,VJ특공대 857회17.05.05. 영덕대게2TV저녁생생정보 19회15.11.10....,"주차, 발렛파킹, 포장, 예약, 무선 인터넷, 반려동물 동반, 유아시설(놀이방), ..."
5,대게 맛집,속초동명항게찜,033-635-1550,강원 속초시 영금정로 33-1지번동명동 1-12 속초동명항게찜,변동대게100그램,매일 10:00 - 22:00,맛있는녀석들 132회17.09.01. 대게찜생방송오늘저녁 389회16.06.24. ...,"단체석, 주차, 발렛파킹, 포장, 배달, 예약, 무선 인터넷, 남/녀 화장실 구분"
6,대게 맛집,영덕강구항대게직판장 대명대게,054-733-0105,경북 영덕군 강구면 강구대게길 13지번강구리 399,변동대게코스요리변동랍스타대게코스요리,매일 09:00 - 22:00연중무휴,None,"단체석, 주차, 발렛파킹, 포장, 예약, 무선 인터넷, 남/녀 화장실 구분, 장애인..."
7,대게 맛집,동명항대게마을,033-638-9770,강원 속초시 영랑해안길 7지번동명동 1-108,"변동대표대게 1kg변동대표킹크랩 1kg15,000원점심특선메뉴 : 물회15,000원...",토요일 10:00 - 23:00일요일 10:00 - 23:00월요일 10:00 - ...,None,"단체석, 주차, 발렛파킹, 포장, 예약"
8,대게 맛집,원식품,033-631-2826,강원 속초시 중앙로 402지번장사동 577-15,,None,None,None
9,대게 맛집,영덕강구항대게직판장 대명대게,054-733-0105,경북 영덕군 강구면 강구대게길 13지번강구리 399,변동대게코스요리변동랍스타대게코스요리,매일 09:00 - 22:00연중무휴,None,"단체석, 주차, 발렛파킹, 포장, 예약, 무선 인터넷, 남/녀 화장실 구분, 장애인..."
